In [1]:
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG')
from vector_store_utils import *
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src')
from utils import *

In [2]:
alll = getData("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md/合并/2.加课外内容.json")
alll[0]

{'id': 'qvrr9a7mfp059p452bfs',
 'title': '# 独立权 管辖权 平等权 自卫权',
 'content': '\n◆ 独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。  \n\n◆ 管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。  \n\n◆ 平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。  \n\n◆ 自卫权，是指主权国家拥有保卫自己的生存和独立的权利。  \n',
 'grade': '选择性必修1',
 'subject': '高中政治',
 'modify_content': '独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。\n\n管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。\n\n平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。\n\n自卫权，是指主权国家拥有保卫自己的生存和独立的权利。',
 '课本内容': '独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。\n\n管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。\n\n平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。\n\n自卫权，是指主权国家拥有保卫自己的生存和独立的权利。',
 '课外内容': '在国际法框架下，主权国家四大基本权利的现实实践具有多维度的体现。以独立权为例，2023年不结盟运动峰会上，印度尼西亚公开拒绝大国干预其镍矿出口政策，援引《联合国宪章》第二条第七款阐明其经济自主权。管辖权在孟晚舟引渡案中得到具象化呈现：加拿大依据属地管辖原则实施司法程序，而中国则启动领事保护机制主张属人管辖权，该案件成为研究国家管辖权竞合冲突的经典案例。平等权的当代实践体现于2022年联合国大会表决俄国决议草案时，圣多美和普林西比（人口21万）与印度（13亿人口）享有同等表决权重，这一制度设计深刻反映了主权平等原则。自卫权的法律边界在2020年伊朗圣城旅指挥官苏莱曼尼遇袭事件中引发全球讨论，多国法学家援引《联合国宪章》第51条，就预防性自卫的合法性

In [3]:
len(alll)

4107

In [4]:
data_type = "从全部数据导入"
import os
if data_type == "从未清洗数据得到":
    subjects = ["高中生物","高中历史","高中地理","高中政治"]
    all_res  = []
    all_text = []
    for sub in subjects:
        paths = get_absolute_file_paths(os.path.join("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/初中",sub),"json")
        for path in paths:
            grade = path.split("/")[-1].split(" ")[-1].split(".")[0]
            res = getData(path)
            new_res = []
            new_text = []
            for item in res:
                new_res.append({"subject":sub,"grade":grade,"title":item['title'].strip(),"content":item['content'].strip()})
                new_text.append(item['content'].strip())
            all_res.extend(new_res)
            all_text.extend(new_text)
else:
    alll = getData("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md/合并/2.加课外内容.json")
    all_res  = []
    all_text = []
    for item in alll:
        all_res.append({"subject":item['subject'].strip(),"grade":item['grade'].strip(),"title":item['title'].strip(),"content":item['课本内容'].strip()})
        all_text.append(item['content'].strip())



In [5]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(
    model_name = "/hpc2hdd/home/fye374/models/BAAI/bge-m3",
    use_fp16=False, 
    device="cuda:0")
dense_dim = ef.dim["dense"]

docs_embeddings = ef(all_text)


/hpc2hdd/home/fye374/.conda/envs/langmanus/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pre tokenize: 100%|██████████| 257/257 [00:01<00:00, 222.22it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 257/257 [00:35<00:00,  7.32it/s]


In [6]:
import sys
sys.path.append("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus")
from src.config.rag import DB_URI,COLLECTION_NAME
from src.RAG.vector_store_utils import *
db_uri = DB_URI
col_name = COLLECTION_NAME
col = get_collection(db_uri,col_name)

collection subjects not found, 请创建collection


In [ ]:
# collection_info = col.describe()
# # collection_info

In [ ]:
# iterator = col.query_iterator(
#     batch_size=10,
#     expr=None,
#     output_fields=["subject"]
# )

# value_set = set()

# while True:
#     result = iterator.next()
#     if not result:
#         iterator.close()
#         break
    
#     for item in result:
#         value_set.add(item["subject"])
# col.release()

In [ ]:
# value_set

In [7]:
col = create_collection(db_uri,col_name, dense_dim)

collection subjects 创建成功，并返回它


In [ ]:
all_res[0]

In [8]:
add_data(col, all_res,docs_embeddings)

100%|██████████| 4107/4107 [00:48<00:00, 84.77it/s]

添加成功


# 检索示例

In [ ]:
print(

In [ ]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(
    model_name = "/hpc2hdd/home/fye374/models/BAAI/bge-m3",
    use_fp16=False, 
    device="cuda:0")

In [ ]:
# query = input("Enter your search query: ")
# print(query)
query = "生物微分子"
query_embeddings = ef([query])


In [ ]:
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG')
from retrieval import *
from reranker import *

In [ ]:
db_uri = "/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG/vector_store/subjects.db"
col_name = "subjects"
col = get_collection(db_uri,col_name)
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"]._getrow(0))
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"]._getrow(0),
    sparse_weight=0.7,
    dense_weight=1.0,
)
# /hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG/vector_store

In [ ]:
rerank(query, hybrid_results)

In [ ]:
hybrid_results